In [65]:
import numpy as np
from random import randint
import networkx as nx
from networkx.algorithms.matching import max_weight_matching
from networkx.algorithms.euler import eulerian_circuit
import itertools
import pickle


In [9]:
cities = np.load("cities.npy", allow_pickle = True)

def distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

dist_matrix = [[distance(c1, c2) for c2 in cities] for c1 in cities]


list

In [19]:
graph = np.array(dist_matrix)

# Christofides algorithm

In [49]:
# http://matejgazda.com/christofides-algorithm-in-python/#:~:text=Christofides%20algorithm%20is%20an%20approximative,the%20distance%20matrix%20are%20symmetric.
def _minimal_spanning_tree_prim(graph, starting_node):
    """
    Args:
        graph: weighted adj. matrix as 2d np.array
        starting_node: node number to start construction of minimal spanning tree
    Returns:
        minimal spanning tree as 2d array calculted by Prim
    """

    node_count = len(graph)
    all_nodes = [i for i in range(node_count)]

    if starting_node is None:
        starting_node = randint(0, node_count-1)

    unvisited_nodes = all_nodes
    visited_nodes = [starting_node]
    unvisited_nodes.remove(starting_node)
    mst = np.zeros((node_count, node_count))

    while len(visited_nodes) != node_count:
        selected_subgraph = graph[np.array(visited_nodes)[:, None], np.array(unvisited_nodes)]
        # we mask non-exist edges with -- so it doesn't crash the argmin
        min_edge_index = np.unravel_index(np.ma.masked_equal(selected_subgraph, 0, copy=False).argmin(),
                                          selected_subgraph.shape)
        edge_from = visited_nodes[min_edge_index[0]]
        edge_to = unvisited_nodes[min_edge_index[1]]
        mst[edge_from, edge_to] = graph[edge_from, edge_to]
        mst[edge_to, edge_from] = graph[edge_from, edge_to]
        unvisited_nodes.remove(edge_to)
        visited_nodes.append(edge_to)
    return mst


def route_cost(graph, path):
    cost = 0
    for index in range(len(path) - 1):
        cost = cost + graph[path[index]][path[index + 1]]
    # add last edge to form a cycle.
    cost = cost + graph[path[-1], path[0]]
    return cost

In [50]:
MST = _minimal_spanning_tree_prim(graph,5)

In [51]:
# Save odd degree vertices into list
odd_vert = []
for i in range(0,1000):
    if (np.count_nonzero(MST[i])%2 == 1):
        odd_vert.append(i)

odd_degree_nodes_ix = np.ix_(odd_vert, odd_vert)

In [55]:
nx_graph = nx.from_numpy_array(-1 * graph[odd_degree_nodes_ix])
matching = nx.max_weight_matching(nx_graph, maxcardinality=True)
euler_multigraph = nx.MultiGraph(MST)
for edge in matching:
    euler_multigraph.add_edge(odd_vert[edge[0]], odd_vert[edge[1]],
                                  weight=graph[odd_vert[edge[0]]][odd_vert[edge[1]]])
euler_tour = list(eulerian_circuit(euler_multigraph, source=5))
path = list(itertools.chain.from_iterable(euler_tour))
path = list(dict.fromkeys(path).keys())
#path.append(5)[:-1]

TypeError: 'NoneType' object is not subscriptable

In [68]:
with open("christofides_path", "wb") as fp:   #Pickling
    pickle.dump(path, fp)

In [69]:
total_dist = 0
for i in range(0,1000):
    total_dist += dist_matrix[path[i]][path[i+1]]

In [70]:
total_dist

26.529455409432007

In [72]:
#path